In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     |████████████████████████████████| 46.9 MB 88 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.8 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595744 sha256=869fe3ef64e828988be3d0b5c5aafccd388751cd112b85db193b40232a5c49a9
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
     |████████████████████████████████| 895 kB 60.6 MB/s 
     |████████████████████████████████| 3.3 MB 60.6 MB/s 
     |████████████████████████████████| 636 kB 66.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
 

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ejav2fs7
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ejav2fs7
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=1b5b7ecdb8bd600d1e7b97d9f8520772f7b8aa3413767bd07667592159a0df0f
  Stored in directory: /tmp/pip-ephem-wheel-cache-rkh4i8zb/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, notebook

import math
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

import pandas as pd

#json 관련 라이브러리
import json

# csv 모듈 import하기
import csv

In [ ]:
#구글 드라이브를 쓰자
from google.colab import drive
#루트를 정해준다.
ROOT = "/content/drive"
print(ROOT)

drive.mount(ROOT) #we mount the google drive at /content/drive

#import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join


MY_GOOGLE_DRIVE_PATH = '/content/drive/My Drive/Colab Notebooks'

print("MY_GOOGLE_DRIVE_PATH: ", MY_GOOGLE_DRIVE_PATH)

%cd "{MY_GOOGLE_DRIVE_PATH}"

/content/drive
Mounted at /content/drive
MY_GOOGLE_DRIVE_PATH:  /content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks


In [ ]:
import torch
torch.__version__

'1.9.0+cu102'

In [ ]:
import transformers
transformers.__version__

'4.11.2'

In [ ]:
import kobert
kobert.__version__

'0.1.2'

In [ ]:
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
# data = pd.read_csv('training_data.csv')
# path = MY_GOOGLE_DRIVE_PATH + "감성대화말뭉치(최종데이터)_Training.xlsx"
# chat_data = pd.read_excel("감성대화말뭉치(최종데이터)_Training.xlsx")
chat_data = pd.read_excel("감성대화.xlsx")
chat_data2 = pd.read_excel("한국어_연속적_대화_데이터셋.xlsx")

In [ ]:
nullcheck1 = chat_data['sentence1'].isnull()
nullcheck2 = chat_data['sentence2'].isnull()
nullcheck3 = chat_data['sentence3'].isnull()
nullcheck4 = chat_data['sentence4'].isnull()

In [ ]:
data_list = []
for check1,check2, check3, check4, label, sentence1, sentence2, sentence3, sentence4 in zip(nullcheck1,nullcheck2, nullcheck3, nullcheck4, chat_data['emotion'], chat_data['sentence1'], chat_data['sentence2'], chat_data['sentence3'], chat_data['sentence4'])  :
    data = []
    sentence = ""

    if check1 != True :
      sentence = sentence1

    if check2 != True :
      sentence = sentence + " " + sentence2

    if check3 != True :
      sentence = sentence + " " + sentence3
    
    if check4 != True :
      sentence = sentence + " " + sentence4
    
    
    data.append(sentence)
    data.append(str(label))

    data_list.append(data)

In [ ]:
data_list[0][0]

'아내가 드디어 출산하게 되어서 정말 신이 나. 아 지금 정말 신이 나. 아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지. '

In [ ]:
for i,value in enumerate(chat_data2['emotion']) :
  chat_data2['emotion'][i] = int(chat_data2['emotion'][i])


nullcheck = chat_data2['sentence'].isnull()
nullcheck1 = chat_data2['emotion'].isnull()

for sen, emo, check, check1 in zip(chat_data2["sentence"], chat_data2["emotion"], nullcheck,nullcheck1) :
  data = []
  data.append(sen)
  data.append(str(emo))
  if emo != 99 :
    if check != True :
      if check1 !=True :
        data_list.append(data)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
sente = []
emo = []
for  i ,set_se in enumerate(data_list) :
  sente.append(set_se[0])
  emo.append(set_se[1])

In [ ]:
# raw_data = {'sentence' : sente,
#             'emotion' : emo}
# raw_data = pd.DataFrame(raw_data)
# raw_data.to_excel(excel_writer='listfile2.xlsx')

In [ ]:
 #분노 => 0
 #슬픔 => 1
 #불안 => 2
 #상처 => 3
 #당황 = 놀람 => 4
 #기쁨 => 5
 #중립 => 6

In [ ]:
# nullcheck = chat_data['talk.content.HS01'].isnull()

In [ ]:
# data_list = []
# for check,label, q in zip(nullcheck,chat_data['profile.emotion.type'], chat_data['talk.content.HS01'])  :
#     data = []
#     data.append(q)
#     data.append(str(label))

#     if check != True :
#       data_list.append(data)

In [ ]:
data_list[0]

['아내가 드디어 출산하게 되어서 정말 신이 나. 아 지금 정말 신이 나. 아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지. ',
 '5']

In [ ]:
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 200
batch_size = 32
warmup_ratio = 0.1
num_epochs = 9
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

86551
9617


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
data_train[0]

(array([   2, 3514, 7794,  862, 6957, 6999,  633,    3,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    

In [ ]:
# for data in train_dataloader :
#     print(data)
#     break

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/2705 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.023655652999878 train acc 0.03125
epoch 1 batch id 201 loss 1.6936911344528198 train acc 0.27238805970149255
epoch 1 batch id 401 loss 1.1101168394088745 train acc 0.38668952618453867
epoch 1 batch id 601 loss 1.2986388206481934 train acc 0.44893926788685523
epoch 1 batch id 801 loss 1.0834060907363892 train acc 0.49801029962546817
epoch 1 batch id 1001 loss 1.1080906391143799 train acc 0.5364323176823177
epoch 1 batch id 1201 loss 0.9956793189048767 train acc 0.5692131557035803
epoch 1 batch id 1401 loss 0.8397714495658875 train acc 0.5940622769450392
epoch 1 batch id 1601 loss 0.6559953689575195 train acc 0.6111414740787008
epoch 1 batch id 1801 loss 1.2184545993804932 train acc 0.6257808162132149
epoch 1 batch id 2001 loss 0.9124975204467773 train acc 0.638055972013993
epoch 1 batch id 2201 loss 0.8289808034896851 train acc 0.6478731258518855
epoch 1 batch id 2401 loss 0.7508483529090881 train acc 0.6566534777176176
epoch 1 batch id 2601 loss 1.049906492233

  0%|          | 0/301 [00:00<?, ?it/s]

epoch 1 test acc 0.7555269200703537


  0%|          | 0/2705 [00:00<?, ?it/s]

...

epoch 8 test acc 0.7532672952902091


  0%|          | 0/2705 [00:00<?, ?it/s]

epoch 9 batch id 1 loss 0.017702633515000343 train acc 1.0
epoch 9 batch id 201 loss 0.16715075075626373 train acc 0.9500932835820896
epoch 9 batch id 401 loss 0.1194334328174591 train acc 0.9531639650872819
epoch 9 batch id 601 loss 0.40715834498405457 train acc 0.9527870216306157
epoch 9 batch id 801 loss 0.2747040390968323 train acc 0.9529104244694132
epoch 9 batch id 1001 loss 0.19144493341445923 train acc 0.9527035464535465
epoch 9 batch id 1201 loss 0.2908368408679962 train acc 0.9527737302248127
epoch 9 batch id 1401 loss 0.11874330788850784 train acc 0.9531584582441114
epoch 9 batch id 1601 loss 0.024816518649458885 train acc 0.9530176452217364
epoch 9 batch id 1801 loss 0.8389854431152344 train acc 0.9533245419211549
epoch 9 batch id 2001 loss 0.16173845529556274 train acc 0.9534607696151924
epoch 9 batch id 2201 loss 0.21262678503990173 train acc 0.9538420036347115
epoch 9 batch id 2401 loss 0.03575456514954567 train acc 0.9539644939608496
epoch 9 batch id 2601 loss 0.3125362

  0%|          | 0/301 [00:00<?, ?it/s]

epoch 9 test acc 0.7546047488762947


In [ ]:
torch.save(model, "model5.pth")
#model4 0 83.5% 94%

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("분노가")
            elif np.argmax(logits) == 1:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 2:
                test_eval.append("불안이")
            elif np.argmax(logits) == 3:
                test_eval.append("상처가")
            elif np.argmax(logits) == 4:
                test_eval.append("당황이")
            elif np.argmax(logits) == 5:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 6:
                test_eval.append("중립")

            print(logits)
        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

using cached model


In [ ]:
#질문 무한반복하기! 0 입력시 종료

end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 춥네요
[-0.87388307 -0.313725   -3.971902   -4.037626    3.5836127  -1.4184761
  5.320851  ]
>> 입력하신 내용에서 중립 느껴집니다.


하고싶은 말을 입력해주세요 : 아 화난다
[ 1.3684901  0.6246823 -2.9096904 -4.7269835 -0.4753533 -1.0563979
  6.1607184]
>> 입력하신 내용에서 중립 느껴집니다.


하고싶은 말을 입력해주세요 : 짜증 나는데요
[ 0.7127633  -0.5618607  -1.6699893  -4.506419    0.55090696 -2.1545322
  6.5447598 ]
>> 입력하신 내용에서 중립 느껴집니다.


하고싶은 말을 입력해주세요 : 꺼져
[ 0.5923695   0.49365377 -3.5040672  -4.466104    2.2071295  -1.7514257
  5.2282577 ]
>> 입력하신 내용에서 중립 느껴집니다.


하고싶은 말을 입력해주세요 : ㅅㅂ 뭐야
[ 1.8696544  0.3602516 -3.7989895 -4.4468246  1.6853995 -1.9512244
  5.293024 ]
>> 입력하신 내용에서 중립 느껴집니다.


하고싶은 말을 입력해주세요 : 아내가 드디어 출산하게 되어서 정말 신이 나. 아 지금 정말 신이 나. 아기가 점점 클게 벌써 기대가 되네
[-1.2220222  -1.7383709  -0.71294713 -0.38803464 -2.4843187   7.7983513
 -1.2643384 ]
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 오늘 비가 오니.. 밖에도 나가기 싫고 아무것도 하기 싫어
[-1.2546414   5.6286387  -2.1887677  -3.2972682  -0.41748488 -2.2248623
  3.7707722 ]
>> 입력하신 내용에서 슬픔이 느껴집

KeyboardInterrupt: ignored